In [20]:
import pandas as pd
import datetime
from time_fix import *
df = pd.read_csv(r"connection_graph.csv")

# change the column 0 name to "id"
df.rename(columns={df.columns[0]: 'id'}, inplace=True)

# clean data
df.drop(columns=['id','company'], inplace=True)
df.drop_duplicates(inplace=True)

# remove rows with same start and end stop
df = df[df['start_stop'] != df['end_stop']]

# apply fix_time
df['departure_time'] = df['departure_time'].apply(fix_time)
df['arrival_time'] = df['arrival_time'].apply(fix_time)

# convert to time
df['departure_time'] = pd.to_datetime(df['departure_time'], format='%H:%M:%S').dt.time
df['arrival_time'] = pd.to_datetime(df['arrival_time'], format='%H:%M:%S').dt.time

# count seconds
df['departure_time_seconds'] = df['departure_time'].apply(lambda x: x.second + x.minute*60 + x.hour*3600)
df['arrival_time_seconds'] = df['arrival_time'].apply(lambda x: x.second + x.minute*60 + x.hour*3600)

# add id to df
df['id'] = range(0, len(df))
df.set_index('id', inplace=True)

df.head()

C:\Users\agabe\AppData\Local\Temp\ipykernel_33616\4221336554.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"connection_graph.csv")


,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_time_seconds,arrival_time_seconds
id,,,,,,,,,,,
0,A,20:52:00,20:53:00,Zajezdnia Obornicka,Paprotna,51.148737,17.021069,51.147752,17.020539,75120,75180
1,A,20:53:00,20:54:00,Paprotna,Obornicka (Wołowska),51.147752,17.020539,51.144385,17.023735,75180,75240
2,A,20:54:00,20:55:00,Obornicka (Wołowska),Bezpieczna,51.144385,17.023735,51.141360,17.026376,75240,75300
3,A,20:55:00,20:57:00,Bezpieczna,Bałtycka,51.141360,17.026376,51.136632,17.030617,75300,75420
4,A,20:57:00,20:59:00,Bałtycka,Broniewskiego,51.136632,17.030617,51.135851,17.037383,75420,75540


In [21]:
# creating list of stops with coordinates, each stop name is unique
stops = df[['start_stop', 'start_stop_lat', 'start_stop_lon']].drop_duplicates(subset=['start_stop'], keep='first').sort_values(by='start_stop')

# droping coordinates from df
df.drop(columns=['start_stop_lat','start_stop_lon', 'end_stop_lat','end_stop_lon'], inplace=True)

stops

,start_stop,start_stop_lat,start_stop_lon
id,,,
76067,8 Maja,51.113669,17.091120
338336,AUCHAN,51.052065,16.974054
255765,Adamczewskich,51.120844,16.869535
213298,Adamieckiego,51.074259,16.962995
35806,Aleja Architektów,51.138819,16.929059
...,...,...,...
434431,Żórawina - Wrocławska,50.986523,17.038588
424992,Żórawina - kościół Św. Trójcy,50.980426,17.039577
424994,Żórawina - osiedle,50.987632,17.012111


In [22]:
import networkx as nx

# create directed graph
graph = nx.DiGraph()

def create_nodes(connection):
    graph.add_node(connection['start_stop'], lat=connection['start_stop_lat'], lon=connection['start_stop_lon'])

stops.apply(create_nodes, axis=1)

print(graph.nodes.data())
print(len(graph.nodes))

[('8 Maja', {'lat': 51.11366919, 'lon': 17.09111971}), ('AUCHAN', {'lat': 51.0520648, 'lon': 16.97405354}), ('Adamczewskich', {'lat': 51.12084396, 'lon': 16.86953486}), ('Adamieckiego', {'lat': 51.07425917, 'lon': 16.96299503}), ('Aleja Architektów', {'lat': 51.13881871, 'lon': 16.92905866}), ('Aleja Pracy', {'lat': 51.0918696, 'lon': 16.98537334}), ('Aleja Wędrowców', {'lat': 51.0879861, 'lon': 16.99188243}), ('Arabska', {'lat': 51.05115765, 'lon': 17.0923256}), ('Arachidowa', {'lat': 51.15417321, 'lon': 16.89901728}), ('Arena', {'lat': 51.094134, 'lon': 17.026404}), ('Arkady (Capitol)', {'lat': 51.1020902, 'lon': 17.02946781}), ('Armii Krajowej', {'lat': 51.086705, 'lon': 17.06947}), ('Armii Krajowej (Bogedaina)', {'lat': 51.08552632, 'lon': 17.06622813}), ('Asfaltowa (szkoła)', {'lat': 51.064267, 'lon': 17.04258}), ('Awicenny', {'lat': 51.08833411, 'lon': 16.95238558}), ('Awicenny (Poczta Polska)', {'lat': 51.08672704, 'lon': 16.94930841}), ('Awicenny (Stacja kolejowa)', {'lat': 51.

In [23]:
# create edges and schedules for edges
def create_edges(connection):
    if not graph.has_edge(connection['start_stop'], connection['end_stop']):
        graph.add_edge(connection['start_stop'], connection['end_stop'], schedule=[])

    # Append the new connection data to the schedule list
    graph[connection['start_stop']][connection['end_stop']]['schedule'].append(connection.to_dict())

df.apply(create_edges, axis=1)

# schedule for example connection, they are different for each direction
for elem in graph['Ostrowskiego']['FAT']['schedule'][0]:
    print(elem + " : " + str(graph['Ostrowskiego']['FAT']['schedule'][0][elem]))

line : 107
departure_time : 07:11:00
arrival_time : 07:12:00
start_stop : Ostrowskiego
end_stop : FAT
departure_time_seconds : 25860
arrival_time_seconds : 25920


In [24]:
# function to retrieve path
def return_path_to_destination(start, end, line, best_routes_df):
    path = []
    (action, line_name, when, stop) = ("wysiąść",line[end]['line'], line[end]['arrival_time'], end)
    path.append((action, line_name, when, stop))
    end = best_routes_df.loc['poprzedni_przystanek', end]
    current_line = line[end]['line']
    while end != start:
        previous_stop = line[best_routes_df.loc['poprzedni_przystanek', end]]

        if previous_stop != None  and previous_stop['line'] != current_line:
            (action, line_name, when, stop) = ("wsiąść", f"{line[end]['line']} ->  {line[end]['end_stop']}", line[end]['departure_time'], line[end]['start_stop'])
            path.append((action, line_name, when, stop))

            tmp_przystanek = best_routes_df.loc['poprzedni_przystanek', end]
            (action,line_name, when, stop) = ("wysiąść", line[tmp_przystanek]['line'], line[tmp_przystanek]['arrival_time'], tmp_przystanek)
            path.append((action, line_name, when, stop))

        future_end = best_routes_df.loc['poprzedni_przystanek', end]

        if future_end != start:
            end = future_end
            current_line = line[end]['line']
        else:
            (action, line_name, when, stop) = ("wsiąść",f"{line[end]['line']} ->  {line[end]['end_stop']}", line[end]['departure_time'], line[end]['start_stop'])
            path.append((action, line_name, when, stop))
            end = future_end
    return path

def print_path(path):
    for i in range(len(path)-1, -1, -1):
        print(path[i])


In [25]:
def count_total_cost(arrival_time_seconds, start_time_seconds):
    return (arrival_time_seconds - start_time_seconds + DAY_SECONDS) % DAY_SECONDS

In [42]:
# %load time_fix.py
from constants import *

# change time to datetime
def fix_time(time_str):
    hours, minutes, seconds = map(int, time_str.split(':'))
    hours = hours % 24
    return f'{hours:02d}:{minutes:02d}:{seconds:02d}'

# convert time to seconds
def time_to_seconds(time):
    return time.second + time.minute*60 + time.hour*3600

def seconds_to_hour_and_minute(seconds):
	return "godziny: ", seconds // HOUR_SECONDS,"minuty: ", (seconds % HOUR_SECONDS) // MIN_SECONDS

In [31]:
import numpy as np
import time as tm
# from time_fix import time_to_seconds


def min_koszt(unchecked_stops, best_routes_df):
    min_koszt = best_routes_df.loc['min_koszt', unchecked_stops[0]]
    min_ulica = unchecked_stops[0]

    for stop in unchecked_stops:
        if best_routes_df.loc['min_koszt', stop] < min_koszt:
            min_koszt = best_routes_df.loc['min_koszt', stop]
            min_ulica = stop
    return min_ulica

def best_connection(schedule, current_time_seconds, current_cost):
    best_con = None
    best_cost = np.inf
    koszt = best_cost

    for connection in schedule:
        czas_oczekiwania = (connection['departure_time_seconds'] - current_time_seconds + DAY_SECONDS) % DAY_SECONDS
        czas_jazdy = ((connection['arrival_time_seconds'] - connection['departure_time_seconds']+ DAY_SECONDS) % DAY_SECONDS)
        koszt = current_cost + czas_oczekiwania + czas_jazdy

        if best_cost > koszt:
            best_cost = koszt
            best_con = connection

    return (best_con, best_cost)

def dijkstra(graph, start, time):
    algorithm_duration = tm.time()
    #dictionary with best connections for each stop
    line = {}
    for node in graph.nodes:
        line[node] = None

    # dataframe with unique stops as columns, it will contain the outcome
    best_routes_df = pd.DataFrame(columns=graph.nodes)
    best_routes_df.loc['min_koszt'] = np.inf
    best_routes_df.loc['poprzedni_przystanek'] = ""
    best_routes_df.loc['arrival_time_seconds'] = None

    # setting initial values
    current_time_seconds = time_to_seconds(time)
    best_routes_df.loc['min_koszt', start] = 0
    best_routes_df.loc['best_arrival_time_seconds',start] = current_time_seconds

    checked_stops = []
    unchecked_stops = list(graph.nodes)
    
    while unchecked_stops:
        #find stop with min cost, if it isn't in unchecked_stops
        current_stop = min_koszt(unchecked_stops, best_routes_df) 

        unchecked_stops.remove(current_stop)
        checked_stops.append(current_stop)

        current_time_seconds = best_routes_df.loc['best_arrival_time_seconds', current_stop]
        current_cost = best_routes_df.loc['min_koszt', current_stop]

        for neighbor in graph.neighbors(current_stop):
            (connection,koszt) = best_connection(graph[current_stop][neighbor]['schedule'], current_time_seconds, current_cost)
            if best_routes_df.loc['min_koszt', neighbor] > koszt:
                best_routes_df.loc['min_koszt', neighbor] = koszt
                best_routes_df.loc['poprzedni_przystanek', neighbor] = current_stop
                best_routes_df.loc['best_arrival_time_seconds',neighbor] = connection['arrival_time_seconds']
                line[neighbor] = connection

    print("Znaleziono trasę")
    algorithm_duration = tm.time() - algorithm_duration
    return (line, best_routes_df,algorithm_duration)



### Przykład 1:
### start = ”Rogowska (P+R)” stop = ”FAT” godzina = 23:30

In [62]:
import sys

start = "PORT LOTNICZY"
stop = "Swojczyce"
start_time = datetime.time(10,8, 0)

(line, best_routes_df,algorithm_duration) = dijkstra(graph, start, start_time)

path = return_path_to_destination(start, stop, line, best_routes_df)
print_path(path)
total_cost = count_total_cost(line[stop]['arrival_time_seconds'], time_to_seconds(start_time))
print("Koszt ",seconds_to_hour_and_minute(total_cost), file=sys.stderr)
print("Czas trwania ",algorithm_duration, file=sys.stderr)

Znaleziono trasę
('wsiąść', '106 ->  Rdestowa', datetime.time(10, 14), 'PORT LOTNICZY')
('wysiąść', 106, datetime.time(10, 27), 'MIŃSKA (Rondo Rotm. Pileckiego)')
('wsiąść', '122 ->  Rogowska (P+R)', datetime.time(10, 27), 'MIŃSKA (Rondo Rotm. Pileckiego)')
('wysiąść', 122, datetime.time(10, 31), 'Strzegomska (krzyżówka)')
('wsiąść', '106 ->  Nowodworska', datetime.time(10, 32), 'Strzegomska (krzyżówka)')
('wysiąść', 106, datetime.time(10, 48), 'Renoma')
('wsiąść', '17 ->  Opera', datetime.time(10, 48), 'Renoma')
('wysiąść', 17, datetime.time(10, 54), 'GALERIA DOMINIKAŃSKA')
('wsiąść', 'D ->  Urząd Wojewódzki (Impart)', datetime.time(10, 55), 'GALERIA DOMINIKAŃSKA')
('wysiąść', 'D', datetime.time(11, 4), 'Kochanowskiego')
('wsiąść', '12 ->  Chopina', datetime.time(11, 4), 'Kochanowskiego')
('wysiąść', 12, datetime.time(11, 11), 'SĘPOLNO')
('wsiąść', '115 ->  Monopolowa', datetime.time(11, 18), 'SĘPOLNO')
('wysiąść', 115, datetime.time(11, 22), 'Swojczyce')


Koszt  ('godziny: ', 1, 'minuty: ', 14)
Czas trwania  67.33264446258545


### Przykład 2:
### start = ”Złotniki”, stop = ”Chełmońskiego”, godzina = 1:40

In [36]:
start = "Złotniki"
stop = "Chełmońskiego"
start_time = datetime.time(1, 40, 0)
(line, best_routes_df,algorithm_duration) = dijkstra(graph, start, start_time)

path = return_path_to_destination(start, stop, line, best_routes_df)
print_path(path)
total_cost = count_total_cost(line[stop]['arrival_time_seconds'], time_to_seconds(start_time))
print("Koszt ",seconds_to_hour_and_minute(total_cost), file=sys.stderr)
print("Czas trwania ",algorithm_duration, file=sys.stderr)

Znaleziono trasę
('wsiąść', '253 ->  Małopolska (Ośrodek dla niewidomych)', datetime.time(1, 46), 'Złotniki')
('wysiąść', 253, datetime.time(2, 16), 'DWORZEC AUTOBUSOWY')
('wsiąść', '247 ->  DWORZEC GŁÓWNY', datetime.time(2, 19), 'DWORZEC AUTOBUSOWY')
('wysiąść', 247, datetime.time(2, 21), 'DWORZEC GŁÓWNY')
('wsiąść', '244 ->  Wzgórze Partyzantów', datetime.time(2, 24), 'DWORZEC GŁÓWNY')
('wysiąść', 244, datetime.time(2, 28), 'GALERIA DOMINIKAŃSKA')
('wsiąść', '241 ->  Urząd Wojewódzki (Impart)', datetime.time(2, 38), 'GALERIA DOMINIKAŃSKA')
('wysiąść', 241, datetime.time(2, 43), 'PL. GRUNWALDZKI')
('wsiąść', '253 ->  Kliniki - Politechnika Wrocławska', datetime.time(2, 44), 'PL. GRUNWALDZKI')
('wysiąść', 253, datetime.time(2, 49), 'Chełmońskiego')


Koszt  ('godziny: ', 1, 'minuty: ', 9)
Czas trwania  37.15615272521973


### Przykład 3:
### start = ”Kliniki - Politechnika Wrocławska”, stop = ”Chełmońskiego”, godzina = 12:40

In [37]:
start = "Kliniki - Politechnika Wrocławska"
stop = "Chełmońskiego"
start_time = datetime.time(12, 40, 0)
(line, best_routes_df,algorithm_duration) = dijkstra(graph, start, start_time)

path = return_path_to_destination(start, stop, line, best_routes_df)
print_path(path)
total_cost = count_total_cost(line[stop]['arrival_time_seconds'], time_to_seconds(start_time))
print("Koszt ",seconds_to_hour_and_minute(total_cost), file=sys.stderr)
print("Czas trwania ",algorithm_duration, file=sys.stderr)

Znaleziono trasę
('wsiąść', '4 ->  Hala Stulecia', datetime.time(12, 41), 'Kliniki - Politechnika Wrocławska')
('wysiąść', 4, datetime.time(12, 46), 'Chełmońskiego')


Koszt  ('godziny: ', 0, 'minuty: ', 6)
Czas trwania  38.05855441093445


In [45]:
# Algorytm A*
from math import sqrt

#constants to count approximate time to destination
v_mpk_km_s = 16.6 / 3600 # average velocity of mpk in Wrocław [km/s]
degree_to_km = 111 #[km] 1 degree is about 111km

def best_connection_min_time(schedule, current_time_seconds, current_cost, previous_connection=None):
    best_con = None
    best_cost = np.inf
    koszt = best_cost

    for connection in schedule:
        czas_oczekiwania = (connection['departure_time_seconds'] - current_time_seconds + 86400) % 86400
        czas_jazdy = ((connection['arrival_time_seconds'] - connection['departure_time_seconds']+ 86400) % 86400)
        koszt = current_cost + czas_oczekiwania + czas_jazdy

        if best_cost > koszt:
            best_cost = koszt
            best_con = connection

    return (best_con, best_cost)

def best_connection_min_transfers(schedule, current_time_seconds, current_cost, previous_connection):
    previous_line = ""
    if previous_connection:
        previous_line = previous_connection['line']
    best_connection = None
    best_cost = np.inf
    koszt = best_cost
    for connection in schedule:
        czas_oczekiwania = (connection['departure_time_seconds'] - current_time_seconds + 86400) % 86400
        czas_jazdy = ((connection['arrival_time_seconds'] - connection['departure_time_seconds']+ 86400) % 86400)
        koszt = current_cost + czas_oczekiwania + czas_jazdy

        if previous_line and connection['line'] != previous_line:
            koszt = koszt + 1000

        if best_cost > koszt:
            best_cost = koszt
            best_connection = connection

    return (best_connection, best_cost)

def h_manhattan(wezel_akutalny, wezel_kocowy):
    # Manhattan distance: s(curr,next) = |curr.x−next.x|+|curr.y−next.y|
    #t = s/v
    s_km = degree_to_km * sqrt((wezel_akutalny['lat']-wezel_kocowy['lat'])**2 + (wezel_akutalny['lon']-wezel_kocowy['lon'])**2)

     #approximate time from stop to end
    return s_km/v_mpk_km_s

def h_euclidean(wezel_akutalny, wezel_kocowy):
    # Euclidean distance: s(curr,next) = sqrt((curr.x−next.x)^2+(curr.y−next.y)^2)
    #t = s/v
    s_km = degree_to_km * (abs(wezel_akutalny['lat']-wezel_kocowy['lat']) + abs(wezel_akutalny['lon']-wezel_kocowy['lon']))
     #approximate time from stop to end
    return s_km/v_mpk_km_s


def astar(graph, start, end, time, h=h_euclidean, best_connection_function=best_connection_min_transfers):
    algorithm_duration = tm.time()

    for node in graph.nodes():
        graph.nodes[node]['g'] = 0
        graph.nodes[node]['h'] = 0
        graph.nodes[node]['f'] = 0

    # creating dataframe with unique stops
    best_routes_df = pd.DataFrame(columns=graph.nodes)

    # adding rows to dataframe
    best_routes_df.loc['min_koszt'] = np.inf #min_koszt czyli f
    best_routes_df.loc['poprzedni_przystanek'] = ""
    best_routes_df.loc['arrival_time_seconds'] = None

    # seeting initial values
    current_time_seconds = time_to_seconds(time)
    best_routes_df.loc['min_koszt', start] = 0
    best_routes_df.loc['best_arrival_time_seconds',start] = current_time_seconds

    checked_stops = [] #zamkniete
    unchecked_stops = [] #otwarte
    unchecked_stops.append(start)

    # creating line with best connections to each stop
    line = {}
    for node in graph.nodes:
        line[node] = None

    while len(unchecked_stops) > 0:
        wezel = None
        koszt_wezla = np.inf
        
        wezel = min_koszt(unchecked_stops, best_routes_df) #zwarca nazwe ulicy
        koszt_wezla = best_routes_df.loc['min_koszt', wezel]
        current_time_seconds = best_routes_df.loc['best_arrival_time_seconds', wezel]

        if wezel == end:
            break
        
        unchecked_stops.remove(wezel)
        checked_stops.append(wezel)

        for neighbor in graph.neighbors(wezel):
            (connection,koszt) = best_connection_function(graph[wezel][neighbor]['schedule'],current_time_seconds, koszt_wezla, line[wezel])

            if best_routes_df.loc['min_koszt', neighbor] > koszt:
                best_routes_df.loc['min_koszt', neighbor] = koszt
                best_routes_df.loc['poprzedni_przystanek', neighbor] = wezel
                best_routes_df.loc['best_arrival_time_seconds',neighbor] = connection['arrival_time_seconds']
                line[neighbor] = connection
 
            if neighbor not in unchecked_stops and neighbor not in checked_stops:
                unchecked_stops.append(neighbor)
                graph.nodes[neighbor]['h'] = h(graph.nodes[neighbor],graph.nodes[end])
                graph.nodes[neighbor]['g'] = graph.nodes[wezel]['g'] + koszt
                graph.nodes[neighbor]['f'] = graph.nodes[neighbor]['g'] + graph.nodes[neighbor]['h']
            elif graph.nodes[neighbor]['g'] > graph.nodes[wezel]['g'] + koszt:
                graph.nodes[neighbor]['g'] = graph.nodes[wezel]['g'] + koszt
                graph.nodes[neighbor]['f'] = graph.nodes[neighbor]['g'] + graph.nodes[neighbor]['h']
                if neighbor in checked_stops:
                    unchecked_stops.append(neighbor)
                    checked_stops.remove(neighbor)
    algorithm_duration = tm.time() - algorithm_duration
    # print("Liczba sprawdzonych przystanków: ",len(checked_stops))
    return (line, best_routes_df,algorithm_duration)

### Kryterium czasu & euklides

In [64]:
# start = "Rogowska (P+R)"
# stop = "FAT"
# start_time = datetime.time(23, 30, 0)

start = "Złotniki"
stop = "Chełmońskiego"
start_time = datetime.time(1, 40, 0)

# start = "Kliniki - Politechnika Wrocławska"
# stop = "Chełmońskiego"
# start_time = datetime.time(12, 40, 0)

(line, astar_best_routes_df,algorithm_duration) = astar(graph, start, stop, start_time, h=h_euclidean,best_connection_function=best_connection_min_time)
path = return_path_to_destination(start, stop, line, astar_best_routes_df)
print_path(path)


total_cost = count_total_cost(line[stop]['arrival_time_seconds'], time_to_seconds(start_time))
print("Koszt ",seconds_to_hour_and_minute(total_cost), file=sys.stderr)
print("Czas trwania ",algorithm_duration, file=sys.stderr)

('wsiąść', '253 ->  Małopolska (Ośrodek dla niewidomych)', datetime.time(1, 46), 'Złotniki')
('wysiąść', 253, datetime.time(2, 16), 'DWORZEC AUTOBUSOWY')
('wsiąść', '247 ->  DWORZEC GŁÓWNY', datetime.time(2, 19), 'DWORZEC AUTOBUSOWY')
('wysiąść', 247, datetime.time(2, 21), 'DWORZEC GŁÓWNY')
('wsiąść', '244 ->  Wzgórze Partyzantów', datetime.time(2, 24), 'DWORZEC GŁÓWNY')
('wysiąść', 244, datetime.time(2, 28), 'GALERIA DOMINIKAŃSKA')
('wsiąść', '241 ->  Urząd Wojewódzki (Impart)', datetime.time(2, 38), 'GALERIA DOMINIKAŃSKA')
('wysiąść', 241, datetime.time(2, 43), 'PL. GRUNWALDZKI')
('wsiąść', '253 ->  Kliniki - Politechnika Wrocławska', datetime.time(2, 44), 'PL. GRUNWALDZKI')
('wysiąść', 253, datetime.time(2, 49), 'Chełmońskiego')


Koszt  ('godziny: ', 1, 'minuty: ', 9)
Czas trwania  5.134310245513916


### Kryterium czasu & manhattan

In [49]:
start = "Rogowska (P+R)"
stop = "FAT"
start_time = datetime.time(23, 30, 0)

# start = "Złotniki"
# stop = "Chełmońskiego"
# start_time = datetime.time(1, 40, 0)

# start = "Kliniki - Politechnika Wrocławska"
# stop = "Chełmońskiego"
# start_time = datetime.time(12, 40, 0)


(line, astar_best_routes_df,algorithm_duration) = astar(graph, start, stop, start_time, h=h_manhattan,best_connection_function=best_connection_min_time)

path = return_path_to_destination(start, stop, line, astar_best_routes_df)
print_path(path)

total_cost = count_total_cost(line[stop]['arrival_time_seconds'], time_to_seconds(start_time))
print("Koszt ",seconds_to_hour_and_minute(total_cost), file=sys.stderr)
print("Czas trwania ",algorithm_duration, file=sys.stderr)

('wsiąść', '23 ->  Strzegomska (krzyżówka)', datetime.time(23, 30), 'Rogowska (P+R)')
('wysiąść', 23, datetime.time(23, 33), 'Nowodworska')
('wsiąść', '106 ->  MUCHOBÓR MAŁY (Stacja kolejowa)', datetime.time(23, 36), 'Nowodworska')
('wysiąść', 106, datetime.time(23, 39), 'Gądowianka')
('wsiąść', '143 ->  Szkocka', datetime.time(23, 41), 'Gądowianka')
('wysiąść', 143, datetime.time(23, 47), 'FAT')


Koszt  ('godziny: ', 0, 'minuty: ', 17)
Czas trwania  0.3169229030609131


### Kryterium przesiadek & euklides

In [65]:
start = "PORT LOTNICZY"
stop = "Swojczyce"
start_time = datetime.time(10, 8, 0)

# start = "Złotniki"
# stop = "Chełmońskiego"
# start_time = datetime.time(1, 40, 0)

# start = "Kliniki - Politechnika Wrocławska"
# stop = "Chełmońskiego"
# start_time = datetime.time(12, 40, 0)


(line, astar_best_routes_df,algorithm_duration) = astar(graph, start, stop, start_time, h=h_euclidean,best_connection_function=best_connection_min_transfers)
path = return_path_to_destination(start, stop, line, astar_best_routes_df)
print_path(path)


total_cost = count_total_cost(line[stop]['arrival_time_seconds'], time_to_seconds(start_time))
print("Koszt ",seconds_to_hour_and_minute(total_cost), file=sys.stderr)
print("Czas trwania ",algorithm_duration, file=sys.stderr)

('wsiąść', '106 ->  Rdestowa', datetime.time(10, 14), 'PORT LOTNICZY')
('wysiąść', 106, datetime.time(10, 48), 'Renoma')
('wsiąść', '17 ->  Opera', datetime.time(10, 48), 'Renoma')
('wysiąść', 17, datetime.time(11, 12), 'SĘPOLNO')
('wsiąść', '115 ->  Monopolowa', datetime.time(11, 18), 'SĘPOLNO')
('wysiąść', 115, datetime.time(11, 22), 'Swojczyce')


Koszt  ('godziny: ', 1, 'minuty: ', 14)
Czas trwania  11.624390363693237


### Kryterium przesiadek & manhattan

In [50]:
start = "Rogowska (P+R)"
stop = "FAT"
start_time = datetime.time(23, 30, 0)

# start = "Tyrmanda"
# stop = "PL. GRUNWALDZKI"
# start_time = datetime.time(8, 40, 0)

# start = "Złotniki"
# stop = "Chełmońskiego"
# start_time = datetime.time(1, 40, 0)

# start = "Kliniki - Politechnika Wrocławska"
# stop = "Chełmońskiego"
# start_time = datetime.time(12, 40, 0)



(line, astar_best_routes_df,algorithm_duration) = astar(graph, start, stop, start_time, h=h_manhattan,best_connection_function=best_connection_min_transfers)

path = return_path_to_destination(start, stop, line, astar_best_routes_df)
print_path(path)

total_cost = count_total_cost(line[stop]['arrival_time_seconds'], time_to_seconds(start_time))
print("Koszt ",seconds_to_hour_and_minute(total_cost), file=sys.stderr)
print("Czas trwania ",algorithm_duration, file=sys.stderr)

('wsiąść', '132 ->  MIŃSKA (Rondo Rotm. Pileckiego)', datetime.time(23, 31), 'Rogowska (P+R)')
('wysiąść', 132, datetime.time(23, 45), 'OPORÓW')
('wsiąść', '4 ->  GRABISZYŃSKA (Cmentarz II)', datetime.time(23, 49), 'OPORÓW')
('wysiąść', 4, datetime.time(23, 53), 'FAT')


Koszt  ('godziny: ', 0, 'minuty: ', 23)
Czas trwania  0.7327852249145508


In [51]:
def get_evereything_about_solution(solution, start, hour, graph, best_connection_function):
	solution_tuple = tuple(solution)
	list_of_lines_of_connections = []
	list_of_lists_of_astar_best_routes_df = []
	
	start_time = hour
	start_stop = start
	
	solution_copy = solution.copy()
	solution_copy.append(start)

	sum_cost = 0

	for stop in solution_copy:
		(line, astar_best_routes_df,algorithm_duration) = astar(graph, start_stop, stop, start_time,h_euclidean,best_connection_function)
		cost = count_total_cost(line[stop]['arrival_time_seconds'], time_to_seconds(start_time))
		sum_cost = sum_cost + cost
		start_time = line[stop]['arrival_time']
		start_stop = stop
		list_of_lines_of_connections.append(line)
		list_of_lists_of_astar_best_routes_df.append(astar_best_routes_df)

	return (sum_cost, list_of_lines_of_connections, list_of_lists_of_astar_best_routes_df)

In [59]:
import random

# create dataframes for each solution with costs
solutions_with_costs = {}


def objective_function(solution, start, hour, graph,best_connection_function):
	solution_tuple = tuple(solution)

	if solution_tuple in solutions_with_costs:
		return solutions_with_costs[solution_tuple]
	start_time = hour
	start_stop = start
	
	solution_copy = solution.copy()
	solution_copy.append(start)

	sum_cost = 0

	for stop in solution_copy:
		(line, astar_best_routes_df,algorithm_duration) = astar(graph, start_stop, stop, start_time,h_euclidean,best_connection_function)
		cost = count_total_cost(line[stop]['arrival_time_seconds'], time_to_seconds(start_time))
		sum_cost = sum_cost + cost
		start_time = line[stop]['arrival_time']
		start_stop = stop

	solutions_with_costs[solution_tuple] = sum_cost
	return sum_cost

# Define the neighborhood function
length = 7

def get_neighbors_sample(neighbors):
	return random.sample(neighbors,length)

def get_neighbors(solution):
	neighbors = []
	for i in range(len(solution)):
		for j in range(i + 1, len(solution)):
			neighbor = solution[:]
			neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
			neighbors.append(neighbor)
	# return neighbors
	return get_neighbors_sample(neighbors)


def tabu_search(initial_solution, max_iterations, tabu_list_size, start, hour, graph, best_connection_function=best_connection_min_transfers):
	algorithm_duration = tm.time()

	best_solution = initial_solution
	current_solution = initial_solution
	tabu_list = []

	for _ in range(max_iterations):
		neighbors = get_neighbors(current_solution)
		best_neighbor = None
		best_neighbor_fitness = float('inf')

		for neighbor in neighbors:
			if neighbor not in tabu_list:
				neighbor_fitness = objective_function(neighbor,start, hour, graph,best_connection_function)
				if neighbor_fitness < best_neighbor_fitness:
					best_neighbor = neighbor
					best_neighbor_fitness = neighbor_fitness

		if best_neighbor is None:
			print("best solution",best_neighbor)
			# No non-tabu neighbors found,
			break

		current_solution = best_neighbor
		tabu_list.append(best_neighbor)
		if len(tabu_list) > tabu_list_size:
			# Remove the oldest entry from the
			tabu_list.pop(0)

		if objective_function(best_neighbor,start, hour, graph,best_connection_function) < objective_function(best_solution,start, hour, graph,best_connection_function):
			# Update the best solution if the
			# current neighbor is better
			best_solution = best_neighbor


		# print(seconds_to_hour_and_minute(solutions_with_costs[tuple(best_solution)]))
	algorithm_duration = tm.time() - algorithm_duration
		
	return (best_solution,solutions_with_costs, algorithm_duration)

### godzina 8:40
### initial_solution = ["Kwiska", "FAT", "PL. GRUNWALDZKI","Złotniki","Młodych Techników"]
#### max_iterations = 8
### tabu_list_size = nieograniczona

In [54]:
start = "Tyrmanda"
hour = datetime.time(8, 40, 0)
initial_solution = ["Kwiska", "FAT", "PL. GRUNWALDZKI","Złotniki","Młodych Techników"]
max_iterations = 8
tabu_list_size = 100_000

(best_solution,solutions_with_costs,algorithm_duration) = tabu_search(initial_solution, max_iterations, tabu_list_size,start, hour, graph, best_connection_function=best_connection_min_transfers)

print("Best solution: {}".format(best_solution))

(sum_cost, list_of_lines_of_connections, list_of_lists_of_astar_best_routes_df) = get_evereything_about_solution(best_solution, start, hour, graph,best_connection_function=best_connection_min_transfers)

best_solution.insert(0,start)
best_solution.append(start)

i = 0

for line, astar_best_routes_df in zip(list_of_lines_of_connections, list_of_lists_of_astar_best_routes_df):
    start = best_solution[i]
    stop = best_solution[i+1]
    print("Start: ",start, "Stop: ",stop)
    path = return_path_to_destination(start, stop, line, astar_best_routes_df)
    print_path(path)
    i += 1


# total_cost = count_total_cost(line[stop]['arrival_time_seconds'], time_to_seconds(start_time))
print("Koszt ",seconds_to_hour_and_minute(sum_cost), file=sys.stderr)
print("Czas trwania ",algorithm_duration, file=sys.stderr)

Best solution: ['FAT', 'PL. GRUNWALDZKI', 'Młodych Techników', 'Kwiska', 'Złotniki']
Start:  Tyrmanda Stop:  FAT
('wsiąść', '107 ->  Zagony', datetime.time(8, 41), 'Tyrmanda')
('wysiąść', 107, datetime.time(8, 50), 'FAT')
Start:  FAT Stop:  PL. GRUNWALDZKI
('wsiąść', '11 ->  Hutmen', datetime.time(8, 51), 'FAT')
('wysiąść', 11, datetime.time(9, 9), 'GALERIA DOMINIKAŃSKA')
('wsiąść', 'D ->  Urząd Wojewódzki (Impart)', datetime.time(9, 9), 'GALERIA DOMINIKAŃSKA')
('wysiąść', 'D', datetime.time(9, 15), 'PL. GRUNWALDZKI')
Start:  PL. GRUNWALDZKI Stop:  Młodych Techników
('wsiąść', '4 ->  most Grunwaldzki', datetime.time(9, 15), 'PL. GRUNWALDZKI')
('wysiąść', 4, datetime.time(9, 18), 'Urząd Wojewódzki (Impart)')
('wsiąść', '13 ->  GALERIA DOMINIKAŃSKA', datetime.time(9, 19), 'Urząd Wojewódzki (Impart)')
('wysiąść', 13, datetime.time(9, 32), 'Młodych Techników')
Start:  Młodych Techników Stop:  Kwiska
('wsiąść', '13 ->  pl. Strzegomski (Muzeum Współczesne)', datetime.time(9, 32), 'Młodych Te

Koszt  ('godziny: ', 1, 'minuty: ', 55)
Czas trwania  432.8340198993683


### godzina 8:40
### initial_solution = ["Kwiska", "FAT", "PL. GRUNWALDZKI","Złotniki","Młodych Techników"]
#### max_iterations = 8
### tabu_list_size = 10

In [58]:
start = "Tyrmanda"
hour = datetime.time(8, 40, 0)
initial_solution = ["Kwiska", "FAT", "PL. GRUNWALDZKI","Złotniki","Młodych Techników"]
max_iterations = 8
tabu_list_size = 10

(best_solution,solutions_with_costs,algorithm_duration) = tabu_search(initial_solution, max_iterations, tabu_list_size,start, hour, graph, best_connection_function=best_connection_min_transfers)

print("Best solution: {}".format(best_solution))

(sum_cost, list_of_lines_of_connections, list_of_lists_of_astar_best_routes_df) = get_evereything_about_solution(best_solution, start, hour, graph,best_connection_function=best_connection_min_transfers)

best_solution.insert(0,start)
best_solution.append(start)

i = 0

for line, astar_best_routes_df in zip(list_of_lines_of_connections, list_of_lists_of_astar_best_routes_df):
    start = best_solution[i]
    stop = best_solution[i+1]
    print("Start: ",start, "Stop: ",stop)
    path = return_path_to_destination(start, stop, line, astar_best_routes_df)
    print_path(path)
    i += 1


# total_cost = count_total_cost(line[stop]['arrival_time_seconds'], time_to_seconds(start_time))
print("Koszt ",seconds_to_hour_and_minute(sum_cost), file=sys.stderr)
print("Czas trwania ",algorithm_duration, file=sys.stderr)

Best solution: ['FAT', 'PL. GRUNWALDZKI', 'Młodych Techników', 'Kwiska', 'Złotniki']
Start:  Tyrmanda Stop:  FAT
('wsiąść', '107 ->  Zagony', datetime.time(8, 41), 'Tyrmanda')
('wysiąść', 107, datetime.time(8, 50), 'FAT')
Start:  FAT Stop:  PL. GRUNWALDZKI
('wsiąść', '11 ->  Hutmen', datetime.time(8, 51), 'FAT')
('wysiąść', 11, datetime.time(9, 9), 'GALERIA DOMINIKAŃSKA')
('wsiąść', 'D ->  Urząd Wojewódzki (Impart)', datetime.time(9, 9), 'GALERIA DOMINIKAŃSKA')
('wysiąść', 'D', datetime.time(9, 15), 'PL. GRUNWALDZKI')
Start:  PL. GRUNWALDZKI Stop:  Młodych Techników
('wsiąść', '4 ->  most Grunwaldzki', datetime.time(9, 15), 'PL. GRUNWALDZKI')
('wysiąść', 4, datetime.time(9, 18), 'Urząd Wojewódzki (Impart)')
('wsiąść', '13 ->  GALERIA DOMINIKAŃSKA', datetime.time(9, 19), 'Urząd Wojewódzki (Impart)')
('wysiąść', 13, datetime.time(9, 32), 'Młodych Techników')
Start:  Młodych Techników Stop:  Kwiska
('wsiąść', '13 ->  pl. Strzegomski (Muzeum Współczesne)', datetime.time(9, 32), 'Młodych Te

Koszt  ('godziny: ', 1, 'minuty: ', 55)
Czas trwania  383.6722755432129


In [60]:
start = "Tyrmanda"
hour = datetime.time(8, 40, 0)
initial_solution = ["Kwiska", "FAT", "PL. GRUNWALDZKI","Złotniki","Młodych Techników"]
max_iterations = 8
tabu_list_size = 10

(best_solution,solutions_with_costs,algorithm_duration) = tabu_search(initial_solution, max_iterations, tabu_list_size,start, hour, graph, best_connection_function=best_connection_min_transfers)

print("Best solution: {}".format(best_solution))

(sum_cost, list_of_lines_of_connections, list_of_lists_of_astar_best_routes_df) = get_evereything_about_solution(best_solution, start, hour, graph,best_connection_function=best_connection_min_transfers)

best_solution.insert(0,start)
best_solution.append(start)

i = 0

for line, astar_best_routes_df in zip(list_of_lines_of_connections, list_of_lists_of_astar_best_routes_df):
    start = best_solution[i]
    stop = best_solution[i+1]
    print("Start: ",start, "Stop: ",stop)
    path = return_path_to_destination(start, stop, line, astar_best_routes_df)
    print_path(path)
    i += 1


# total_cost = count_total_cost(line[stop]['arrival_time_seconds'], time_to_seconds(start_time))
print("Koszt ",seconds_to_hour_and_minute(sum_cost), file=sys.stderr)
print("Czas trwania ",algorithm_duration, file=sys.stderr)

Best solution: ['FAT', 'PL. GRUNWALDZKI', 'Młodych Techników', 'Kwiska', 'Złotniki']
Start:  Tyrmanda Stop:  FAT
('wsiąść', '107 ->  Zagony', datetime.time(8, 41), 'Tyrmanda')
('wysiąść', 107, datetime.time(8, 50), 'FAT')
Start:  FAT Stop:  PL. GRUNWALDZKI
('wsiąść', '11 ->  Hutmen', datetime.time(8, 51), 'FAT')
('wysiąść', 11, datetime.time(9, 9), 'GALERIA DOMINIKAŃSKA')
('wsiąść', 'D ->  Urząd Wojewódzki (Impart)', datetime.time(9, 9), 'GALERIA DOMINIKAŃSKA')
('wysiąść', 'D', datetime.time(9, 15), 'PL. GRUNWALDZKI')
Start:  PL. GRUNWALDZKI Stop:  Młodych Techników
('wsiąść', '4 ->  most Grunwaldzki', datetime.time(9, 15), 'PL. GRUNWALDZKI')
('wysiąść', 4, datetime.time(9, 18), 'Urząd Wojewódzki (Impart)')
('wsiąść', '13 ->  GALERIA DOMINIKAŃSKA', datetime.time(9, 19), 'Urząd Wojewódzki (Impart)')
('wysiąść', 13, datetime.time(9, 32), 'Młodych Techników')
Start:  Młodych Techników Stop:  Kwiska
('wsiąść', '13 ->  pl. Strzegomski (Muzeum Współczesne)', datetime.time(9, 32), 'Młodych Te

Koszt  ('godziny: ', 1, 'minuty: ', 55)
Czas trwania  205.35099172592163
